# Feature Engineering Calculations

## 1. Temporal Features
```markdown
# Using 'Timestamp' column
- hour_of_day = extract hour from Timestamp (0-23)
- day_of_week = extract weekday from Timestamp (0-6)
- week_of_year = extract week from Timestamp (1-52)
- month = extract month from Timestamp (1-12)
- is_weekend = day_of_week in [5,6]
- is_business_hours = hour_of_day between 9 and 17

# Using 'Timestamp' + grouping
- incidents_last_24h = count(AlertId) where Timestamp within last 24h grouped by OrgId
- incidents_last_7d = count(AlertId) where Timestamp within last 7d grouped by OrgId
- time_since_last_incident = current_timestamp - last(Timestamp) grouped by OrgId
```

## 2. Categorical Aggregations
```markdown
# Organization Level
- org_incident_count = count(AlertId) grouped by OrgId
- org_detector_count = count(distinct DetectorId) grouped by OrgId
- org_category_dist = count(AlertId) grouped by (OrgId, Category) / org_incident_count

# Detector Performance
- detector_alert_freq = count(AlertId) grouped by DetectorId
- detector_true_positive_rate = count(where IncidentGrade='TruePositive') / total_alerts per DetectorId
- detector_false_positive_rate = count(where IncidentGrade='FalsePositive') / total_alerts per DetectorId
```

## 3. Location-Based Features
```markdown
# Using CountryCode, State, City
- location_incident_rate = count(AlertId) grouped by (CountryCode, State, City)
- country_incident_rate = count(AlertId) grouped by CountryCode
- geographic_risk_score = location_incident_rate / avg(location_incident_rate)
```

## 4. Entity-Based Features
```markdown
# Device Features
- device_incident_count = count(AlertId) grouped by DeviceId
- device_risk_score = count(where IncidentGrade='TruePositive') / device_incident_count

# Account Features
- account_incident_count = count(AlertId) grouped by AccountSid
- account_risk_score = count(where IncidentGrade='TruePositive') / account_incident_count
```

## 5. Technical Indicators
```markdown
# OS Features
- os_risk_score = count(where IncidentGrade='TruePositive') grouped by (OSFamily, OSVersion)

# Application Features
- app_risk_score = count(where IncidentGrade='TruePositive') grouped by ApplicationId

# Security Indicators
- ip_risk_score = count(where IncidentGrade='TruePositive') grouped by IpAddress
- url_risk_score = count(where IncidentGrade='TruePositive') grouped by Url
```

## 6. Alert Chain Features
```markdown
# Using AlertId, Timestamp
- related_alerts = count(AlertId) grouped by IncidentId
- alert_sequence = rank(Timestamp) within IncidentId
- time_between_alerts = Timestamp - lag(Timestamp) within IncidentId
```

## 7. Severity Indicators
```markdown
# Using IncidentGrade
- severity_score = case(
    TruePositive = 1.0,
    BenignPositive = 0.5,
    FalsePositive = 0.0
)
- org_risk_level = avg(severity_score) grouped by OrgId
```

## 8. Behavioral Patterns
```markdown
# Using EntityType, EvidenceRole
- entity_behavior_score = count(AlertId) grouped by (EntityType, EvidenceRole)
- entity_pattern = sequence(EntityType) within IncidentId
```

## 9. Text-Based Features
```markdown
# Using AlertTitle
- title_tokens = tokenize(AlertTitle)
- title_category = extract_category(AlertTitle)
```

## 10. Compliance & Policy Features
```markdown
# Using Category, IncidentGrade
- control_effectiveness = 1 - (count(where IncidentGrade='FalsePositive') / total_alerts)
```

## 11. Incident Grade Features
```markdown
# Using IncidentGrade
- true_positive_rate = count(where IncidentGrade='TruePositive') / total_alerts
- false_positive_rate = count(where IncidentGrade='FalsePositive') / total_alerts
- grade_by_category = pivot_table(IncidentGrade, Category)
```

## 12. Evidence Role Features
```markdown
# Using EvidenceRole
- role_distribution = count(AlertId) grouped by EvidenceRole
- role_severity = avg(severity_score) grouped by EvidenceRole
```

## Key Aggregation Windows
```markdown
# Time Windows
- 24 hours
- 7 days
- 30 days

# Grouping Levels
- Per AlertId
- Per IncidentId
- Per OrgId
- Per DetectorId
- Per EntityType
```


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Read the data
df = pd.read_csv('/Users/lukishyadav/Desktop/Engineering/case_study/ai_strike/data/data_head/df_head_500.csv')

# Convert timestamp to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [10]:
def create_entity_features(df):
    """
    Creates features based on entity relationships and history
    """
    features = pd.DataFrame()
    features['AlertId'] = df['AlertId']
    
    # Copy the columns we need for merging
    features['EntityType'] = df['EntityType']
    features['EvidenceRole'] = df['EvidenceRole']
    features['IpAddress'] = df['IpAddress']
    features['Url'] = df['Url']
    
    # Entity Type and Evidence Role features
    entity_stats = df.groupby('EntityType').agg({
        'AlertId': 'count',
        'IncidentGrade': lambda x: (x == 'TruePositive').mean()
    }).rename(columns={
        'AlertId': 'entity_alert_count',
        'IncidentGrade': 'entity_true_positive_rate'
    })
    
    evidence_stats = df.groupby('EvidenceRole').agg({
        'AlertId': 'count',
        'IncidentGrade': lambda x: (x == 'TruePositive').mean()
    }).rename(columns={
        'AlertId': 'evidence_alert_count',
        'IncidentGrade': 'evidence_true_positive_rate'
    })
    
    # IP Address features
    ip_stats = df[df['IpAddress'].notna()].groupby('IpAddress').agg({
        'AlertId': 'count',
        'IncidentGrade': lambda x: (x == 'TruePositive').mean()
    }).rename(columns={
        'AlertId': 'ip_alert_count',
        'IncidentGrade': 'ip_true_positive_rate'
    })
    
    # URL features
    url_stats = df[df['Url'].notna()].groupby('Url').agg({
        'AlertId': 'count',
        'IncidentGrade': lambda x: (x == 'TruePositive').mean()
    }).rename(columns={
        'AlertId': 'url_alert_count',
        'IncidentGrade': 'url_true_positive_rate'
    })
    
    # Merge features back
    features = (features
        .merge(entity_stats, 
               left_on='EntityType',
               right_index=True,
               how='left')
        .merge(evidence_stats,
               left_on='EvidenceRole',
               right_index=True,
               how='left')
        .merge(ip_stats,
               left_on='IpAddress',
               right_index=True,
               how='left')
        .merge(url_stats,
               left_on='Url',
               right_index=True,
               how='left')
    )
    
    # Drop the columns we used for merging
    features = features.drop(['EntityType', 'EvidenceRole', 'IpAddress', 'Url'], axis=1)
    
    return features

def create_per_alert_features(df):
    """
    Creates features that can be calculated directly from each alert record
    """
    features = pd.DataFrame()
    features['AlertId'] = df['AlertId']
    
    # Temporal features per alert
    features['hour'] = df['Timestamp'].dt.hour
    features['day_of_week'] = df['Timestamp'].dt.dayofweek
    features['week'] = df['Timestamp'].dt.isocalendar().week
    features['month'] = df['Timestamp'].dt.month
    features['is_weekend'] = features['day_of_week'].isin([5,6]).astype(int)
    features['is_business_hours'] = ((features['hour'] >= 9) & 
                                   (features['hour'] < 17)).astype(int)
    
    # MITRE features
    features['mitre_technique_count'] = df['MitreTechniques'].fillna('').str.count(';') + 1
    features['has_mitre'] = df['MitreTechniques'].notna().astype(int)
    
    # Entity features
    features['entity_type'] = pd.Categorical(df['EntityType']).codes
    features['evidence_role'] = pd.Categorical(df['EvidenceRole']).codes
    
    # Location features
    features['has_location'] = df[['CountryCode', 'State', 'City']].notna().any(axis=1).astype(int)
    
    # Technical features
    features['has_ip'] = df['IpAddress'].notna().astype(int)
    features['has_url'] = df['Url'].notna().astype(int)
    features['has_sha256'] = df['Sha256'].notna().astype(int)
    features['has_registry'] = df[['RegistryKey', 'RegistryValueName', 'RegistryValueData']].notna().any(axis=1).astype(int)
    
    # Category and detector
    features['category_code'] = pd.Categorical(df['Category']).codes
    features['detector_code'] = pd.Categorical(df['DetectorId']).codes
    
    return features

def create_time_window_features(df, windows=[1, 7, 30]):
    """
    Creates features based on different time windows
    windows: list of days for looking back
    """
    features = pd.DataFrame()
    features['AlertId'] = df['AlertId']
    
    # Sort by timestamp
    df = df.sort_values('Timestamp')
    
    for days in windows:
        # Convert days to hours for more granular windows
        hours = days * 24
        
        for group in ['OrgId', 'Category', 'DetectorId', 'EntityType']:
            col_name = f'prev_{days}d_{group}_count'
            
            # Group and calculate rolling counts
            grouped = df.groupby(group)
            
            # Calculate time difference in hours
            temp_df = pd.DataFrame()
            for name, group_data in grouped:
                group_data = group_data.copy()
                group_data['rolling_count'] = (
                    group_data.assign(count=1)
                    .set_index('Timestamp')['count']
                    .rolling(f'{hours}H')
                    .sum()
                )
                temp_df = pd.concat([temp_df, group_data])
            
            # Merge the counts back
            features[col_name] = (
                temp_df.sort_index()['rolling_count']
                .fillna(0)
            )
    
    return features

def create_all_features(data_path):
    """
    Main function to create all features
    """
    # Read data
    print("Reading data...")
    df = pd.read_csv(data_path)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    
    # Create different feature sets
    print("Creating per-alert features...")
    alert_features = create_per_alert_features(df)
    
    print("Creating time window features...")
    time_features = create_time_window_features(df)
    
    print("Creating entity features...")
    entity_features = create_entity_features(df)
    
    # Combine all features
    print("Combining all features...")
    all_features = alert_features.merge(
        time_features, 
        on='AlertId', 
        how='left'
    ).merge(
        entity_features,
        on='AlertId',
        how='left'
    )
    
    return all_features

# Usage
features = create_all_features('/Users/lukishyadav/Desktop/Engineering/case_study/ai_strike/data/data_head/df_head_500.csv')
print("Features shape:", features.shape)
print("\nFeature columns:", features.columns.tolist())

Reading data...
Creating per-alert features...
Creating time window features...


/var/folders/qd/kg7p9vf14yvg21rbd8y_sp_r0000gn/T/ipykernel_34585/1943326059.py:139: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  group_data.assign(count=1)
/var/folders/qd/kg7p9vf14yvg21rbd8y_sp_r0000gn/T/ipykernel_34585/1943326059.py:139: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  group_data.assign(count=1)
/var/folders/qd/kg7p9vf14yvg21rbd8y_sp_r0000gn/T/ipykernel_34585/1943326059.py:139: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  group_data.assign(count=1)
/var/folders/qd/kg7p9vf14yvg21rbd8y_sp_r0000gn/T/ipykernel_34585/1943326059.py:139: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  group_data.assign(count=1)
/var/folders/qd/kg7p9vf14yvg21rbd8y_sp_r0000gn/T/ipykernel_34585/1943326059.py:139: FutureWarning: 'H' is deprecated and will be removed in a future version

Creating entity features...
Combining all features...
Features shape: (500, 38)

Feature columns: ['AlertId', 'hour', 'day_of_week', 'week', 'month', 'is_weekend', 'is_business_hours', 'mitre_technique_count', 'has_mitre', 'entity_type', 'evidence_role', 'has_location', 'has_ip', 'has_url', 'has_sha256', 'has_registry', 'category_code', 'detector_code', 'prev_1d_OrgId_count', 'prev_1d_Category_count', 'prev_1d_DetectorId_count', 'prev_1d_EntityType_count', 'prev_7d_OrgId_count', 'prev_7d_Category_count', 'prev_7d_DetectorId_count', 'prev_7d_EntityType_count', 'prev_30d_OrgId_count', 'prev_30d_Category_count', 'prev_30d_DetectorId_count', 'prev_30d_EntityType_count', 'entity_alert_count', 'entity_true_positive_rate', 'evidence_alert_count', 'evidence_true_positive_rate', 'ip_alert_count', 'ip_true_positive_rate', 'url_alert_count', 'url_true_positive_rate']


/var/folders/qd/kg7p9vf14yvg21rbd8y_sp_r0000gn/T/ipykernel_34585/1943326059.py:139: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  group_data.assign(count=1)
/var/folders/qd/kg7p9vf14yvg21rbd8y_sp_r0000gn/T/ipykernel_34585/1943326059.py:139: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  group_data.assign(count=1)
/var/folders/qd/kg7p9vf14yvg21rbd8y_sp_r0000gn/T/ipykernel_34585/1943326059.py:139: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  group_data.assign(count=1)
/var/folders/qd/kg7p9vf14yvg21rbd8y_sp_r0000gn/T/ipykernel_34585/1943326059.py:139: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  group_data.assign(count=1)
/var/folders/qd/kg7p9vf14yvg21rbd8y_sp_r0000gn/T/ipykernel_34585/1943326059.py:139: FutureWarning: 'H' is deprecated and will be removed in a future version

In [11]:
features

,AlertId,hour,day_of_week,week,month,is_weekend,is_business_hours,mitre_technique_count,has_mitre,entity_type,...,prev_30d_DetectorId_count,prev_30d_EntityType_count,entity_alert_count,entity_true_positive_rate,evidence_alert_count,evidence_true_positive_rate,ip_alert_count,ip_true_positive_rate,url_alert_count,url_true_positive_rate
0,123247,6,1,23,6,0,0,1,0,5,...,0.0,0.0,97,0.381443,245,0.404082,1,1.000000,468,0.354701
1,210035,3,4,24,6,0,0,1,0,14,...,0.0,0.0,115,0.460870,255,0.298039,403,0.342432,468,0.354701
2,712507,4,3,24,6,0,0,1,1,13,...,0.0,0.0,32,0.281250,245,0.404082,403,0.342432,1,0.000000
3,774301,16,0,24,6,0,1,1,0,13,...,0.0,0.0,32,0.281250,245,0.404082,403,0.342432,1,0.000000
4,188041,1,5,24,6,1,0,1,0,14,...,0.0,0.0,115,0.460870,255,0.298039,403,0.342432,468,0.354701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,805091,13,6,23,6,1,1,2,1,14,...,0.0,0.0,115,0.460870,255,0.298039,403,0.342432,468,0.354701
496,495820,19,4,23,6,0,0,2,1,5,...,0.0,0.0,97,0.381443,245,0.404082,1,1.000000,468,0.354701
497,5525,8,2,24,6,0,0,1,0,8,...,0.0,0.0,60,0.066667,255,0.298039,403,0.342432,468,0.354701
498,271918,18,0,23,6,0,0,1,0,6,...,0.0,0.0,30,0.200000,255,0.298039,403,0.342432,468,0.354701


In [12]:
df.columns

Index(['Id', 'OrgId', 'IncidentId', 'AlertId', 'Timestamp', 'DetectorId',
       'AlertTitle', 'Category', 'MitreTechniques', 'IncidentGrade',
       'ActionGrouped', 'ActionGranular', 'EntityType', 'EvidenceRole',
       'DeviceId', 'Sha256', 'IpAddress', 'Url', 'AccountSid', 'AccountUpn',
       'AccountObjectId', 'AccountName', 'DeviceName', 'NetworkMessageId',
       'EmailClusterId', 'RegistryKey', 'RegistryValueName',
       'RegistryValueData', 'ApplicationId', 'ApplicationName',
       'OAuthApplicationId', 'ThreatFamily', 'FileName', 'FolderPath',
       'ResourceIdName', 'ResourceType', 'Roles', 'OSFamily', 'OSVersion',
       'AntispamDirection', 'SuspicionLevel', 'LastVerdict', 'CountryCode',
       'State', 'City'],
      dtype='object')

In [13]:
# Group by AlertId and count unique IncidentGrades
alert_grade_counts = df.groupby('AlertId')['IncidentGrade'].nunique()

# Find alerts with multiple grades
multiple_grades = alert_grade_counts[alert_grade_counts > 1]

print(f"Number of AlertIds with multiple IncidentGrades: {len(multiple_grades)}")
print("\nDistribution of number of grades per AlertId:")
print(alert_grade_counts.value_counts().sort_index())

# Optional: Look at some examples of alerts with multiple grades
if len(multiple_grades) > 0:
    print("\nExample alerts with multiple grades:")
    print(df[df['AlertId'].isin(multiple_grades.index)][['AlertId', 'IncidentGrade']])

Number of AlertIds with multiple IncidentGrades: 0

Distribution of number of grades per AlertId:
IncidentGrade
0      3
1    497
Name: count, dtype: int64
